In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
model1 = SentenceTransformer('./tuning1/')
#ns = pd.read_csv('sor_test.csv')
#ns = pd.read_excel('네이버_뉴스_제목_데이터.xlsx', engine='openpyxl')
#ns = ns.drop_duplicates(['Field2_text'], keep='first')
#text = ns['Field2_text'].tolist()
ns = pd.read_excel('네이버 뉴스_(10).xlsx', engine='openpyxl')
ns1 = pd.read_excel('네이버 뉴스_(11).xlsx', engine='openpyxl')
ns = ns.append(ns1, ignore_index = True)
ns = ns.fillna('')
ns = ns[ns['Text'] != '']
ns = ns.drop_duplicates(['Text'], keep='first')
ns.sort_values(by=['Text1'],inplace=True)
ns.reset_index(inplace=True)
text = ns['Text'].tolist()
ns

,index,Text,Text1
0,1173,바다가 일자리다…'2021 해양수산 온라인 취업박람회' 30일 개막,"""2021 해양수산 취업박람회"" 30일부터 온라인 진행"
1,1182,"일자리박람회 오프라인 면접, 다음 달 시작","""2021 해양수산 취업박람회"" 30일부터 온라인 진행"
2,1177,경북 온·오프라인 채용박람회 개최,"""2021 해양수산 취업박람회"" 30일부터 온라인 진행"
3,1176,강원일자리박람회 내달 2일 대면 면접 돌입,"""2021 해양수산 취업박람회"" 30일부터 온라인 진행"
4,1174,"안양시, 온·오프라인 연계 릴레이 일자리박람회","""2021 해양수산 취업박람회"" 30일부터 온라인 진행"
...,...,...,...
1713,906,희망회복자금 2차 신속지급 시작… 61만개社·1조원 규모,희망회복자금 2차 신속지급 시작
1714,903,희망회복자금 2차 신속지급 개시…61만여곳에 1조원 지원,희망회복자금 2차 신속지급 시작
1715,897,희망회복자금 2차 지급 시작…61.1만개사에 1조 지원,희망회복자금 2차 신속지급 시작
1716,923,희망회복자금 2차 신속지급 30일 개시…61만개사·1조원 규모,희망회복자금 2차 신속지급 시작


In [2]:
ns['tag_i'] = ''
ns.at[0, 'tag_i'] = 1
for i in ns.index:
    if i != 0:
        ns.at[i, 'tag_i'] = ns.loc[i - 1, 'tag_i']
        if ns.loc[i - 1, 'Text1'] != ns.loc[i, 'Text1']:
            ns.at[i, 'tag_i'] = ns.loc[i, 'tag_i'] + 1
tag = ns['tag_i'].tolist()

In [3]:
#fine tuning
ns1 = pd.read_excel('네이버 뉴스_(12).xlsx', engine='openpyxl')
ns1 = ns1.fillna('')
ns1 = ns1[ns1['Text'] != '']
ns1 = ns1.drop_duplicates(['Text'], keep='first')
ns1.sort_values(by=['Text1'],inplace=True)
ns1.reset_index(inplace=True)
ns1['tag_i'] = ''
ns1.at[0, 'tag_i'] = 1
for i in ns1.index:
    if i != 0:
        ns1.at[i, 'tag_i'] = ns1.loc[i - 1, 'tag_i']
        if ns1.loc[i - 1, 'Text1'] != ns1.loc[i, 'Text1']:
            ns1.at[i, 'tag_i'] = ns1.loc[i, 'tag_i'] + 1
#ns1.to_csv('ns1.csv', encoding='utf-8-sig')

In [4]:
#Compute embeddings
embeddings = model.encode(text, convert_to_tensor=True)
embeddings1 = model1.encode(text, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores1 = util.pytorch_cos_sim(embeddings1, embeddings1)
cosine_scores1 = np.round(cosine_scores1.cpu(),4)
cosine_scores2 = util.pytorch_cos_sim(embeddings, embeddings)
cosine_scores2 = np.round(cosine_scores2.cpu(),4)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=500)
tfvect = vect.fit(text)

tfidv_df = pd.DataFrame(tfvect.transform(text).toarray(), columns = sorted(vect.vocabulary_))
#tfidv_df

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
tf_res = cosine_similarity(tfidv_df, tfidv_df)
tr_res = np.round(tf_res,4)

In [7]:
pa_ac = []
for i in range(len(cosine_scores1)-1):
    for j in range(i+1, len(cosine_scores1)):
        pa_ac.append([text[i],text[j],tag[i],tag[j], np.array(cosine_scores1[i][j]), np.array(cosine_scores2[i][j]),tr_res[i][j]])

In [8]:
pairs1, pairs2, pairs3 = [],[],[]
for i in range(len(cosine_scores1)):
    i_s = cosine_scores1[i].tolist()
    i_s = {ii:score for ii, score in enumerate(i_s)}
    del i_s[i]
    i_s = sorted(i_s.items(), key=lambda x : x[1], reverse=True)    
    pairs1.append({'index': [i,i_s[0][0]], 'score':i_s[0][1] })
    
    i_s = cosine_scores2[i].tolist()
    i_s = {ii:score for ii, score in enumerate(i_s)}
    del i_s[i]
    i_s = sorted(i_s.items(), key=lambda x : x[1], reverse=True)    
    pairs2.append({'index': [i,i_s[0][0]], 'score':i_s[0][1] })
    
    i_tf = tf_res[i]
    i_tf = {ii:score for ii, score in enumerate(i_tf)}
    del i_tf[i]
    i_tf = sorted(i_tf.items(), key=lambda x : x[1], reverse=True)    
    pairs3.append({'index': [i,i_tf[0][0]], 'score':i_tf[0][1] })

In [9]:
"""
def sens_cal(ns, idx, cut):
    t_df = pd.DataFrame()
    t_df['sco'] = cosine_scores[i].tolist()
    t_df['sco'] = t_df['sco'].apply(lambda x : x if x >= cut else float('inf'))
    t_df.at[idx, 'sco'] = float('inf')
    t_df['tag'] = ns['tag_i']
    # cut을 넘긴 텍스트 갯수
    c = len(t_df[t_df['sco'] != float('inf')])
    # cut을 넘긴 텍스트 중에 맞는것
    if c == 0:
        ct = 0
        pr = 0
    else:
        ct = len(t_df[(t_df['sco'] != float('inf')) & (t_df['tag'] == t_df.loc[idx, 'tag'])])
        pr = round(ct/c*100,2)
    # 같은 태그의 문장 개수
    tg1 = len(t_df[t_df['tag'] == t_df.loc[idx, 'tag']])
    #민감도 cut을 넘긴 텍스트중 tag가 맞는것/ 실제 tag에서 문장 개수
    #정밀도 cut을 넘긴 텍스트중 tag가 맞는것/ cut을 넘긴 텍스트 개수
    return [ct, tg1, round(ct/tg1*100,2) , ct, c, pr]
    
sen6,sen7,sen8  =[],[],[]
for i in range(len(cosine_scores)):
    sen6.append(sens_cal(ns, i, 0.6))
    sen7.append(sens_cal(ns, i, 0.7))
    sen8.append(sens_cal(ns, i, 0.8))
"""

"\ndef sens_cal(ns, idx, cut):\n    t_df = pd.DataFrame()\n    t_df['sco'] = cosine_scores[i].tolist()\n    t_df['sco'] = t_df['sco'].apply(lambda x : x if x >= cut else float('inf'))\n    t_df.at[idx, 'sco'] = float('inf')\n    t_df['tag'] = ns['tag_i']\n    # cut을 넘긴 텍스트 갯수\n    c = len(t_df[t_df['sco'] != float('inf')])\n    # cut을 넘긴 텍스트 중에 맞는것\n    if c == 0:\n        ct = 0\n        pr = 0\n    else:\n        ct = len(t_df[(t_df['sco'] != float('inf')) & (t_df['tag'] == t_df.loc[idx, 'tag'])])\n        pr = round(ct/c*100,2)\n    # 같은 태그의 문장 개수\n    tg1 = len(t_df[t_df['tag'] == t_df.loc[idx, 'tag']])\n    #민감도 cut을 넘긴 텍스트중 tag가 맞는것/ 실제 tag에서 문장 개수\n    #정밀도 cut을 넘긴 텍스트중 tag가 맞는것/ cut을 넘긴 텍스트 개수\n    return [ct, tg1, round(ct/tg1*100,2) , ct, c, pr]\n    \nsen6,sen7,sen8  =[],[],[]\nfor i in range(len(cosine_scores)):\n    sen6.append(sens_cal(ns, i, 0.6))\n    sen7.append(sens_cal(ns, i, 0.7))\n    sen8.append(sens_cal(ns, i, 0.8))\n"

In [10]:
#pairs2 = sorted(pairs2, key=lambda x: x['score'], reverse=True)
tn_i, t1, t2, t3 = [],[],[],[]
t_i, t_j1, t_j2, t_j3 = [],[],[],[]
tn_i, tn_j1, tn_j2, tn_j3 = [],[],[],[]
ti_i, ti_j1, ti_j2, ti_j3 = [],[],[],[]
score1, score2, score3 = [], [], []
for pair in pairs1:
    i1, j1 = pair['index']    
    t_i.append(text[i1])
    tn_i.append(i)
    ti_i.append(tag[i1])
    
    t1.append(text[j1])
    tn_j1.append(j1)
    ti_j1.append(tag[j1])
        
    score1.append(np.array(pair['score']))
    
for pair in pairs2:
    i2, j2 = pair['index']  
    t2.append(text[j2])
    tn_j2.append(j2)
    ti_j2.append(tag[j2])
    
    score2.append(np.array(pair['score']))

In [11]:
for pair in pairs3:
    i3, j3 = pair['index']
    t3.append(text[j3])
    tn_j3.append(j3)
    ti_j3.append(tag[j3])
    score3.append(np.array(pair['score']))

In [12]:
ff1 =pd.DataFrame()
ff1['text'] = t_i
ff1['text1'] = t1
ff1['text2'] = t2
ff1['text3'] = t3
#ff1['text3'] = t3
ff1['tag'] = ti_i
ff1['tag1'] = ti_j1
ff1['tag2'] = ti_j2
ff1['tag3'] = ti_j3
#ff1['tag3'] = ti_j3
ff1['score1'] = score1
ff1['score2'] = score2
ff1['score3'] = score3
#ff1['score3'] = score3
ff1['num_text'] = tn_i
ff1['num_text1'] = tn_j1
ff1['num_text2'] = tn_j2
ff1['num_text3'] = tn_j3

"""
#ff1['num_text3'] = tn_j3
sen6 = pd.DataFrame(sen6)
col6 = sen6.columns.tolist()
sen7 = pd.DataFrame(sen7)
col7 = sen7.columns.tolist()
sen8 = pd.DataFrame(sen8)
col8 = sen8.columns.tolist()

ff1['sensitivity(0.6)'] = sen6[col6[2]]
ff1['sensitivity(0.7)'] = sen7[col7[2]]
ff1['sensitivity(0.8)'] = sen8[col8[2]]
ff1[' '] = ''
ff1['precision(0.6)'] = sen6[col6[5]]
ff1['precision(0.7)'] = sen7[col7[5]]
ff1['precision(0.8)'] = sen8[col8[5]]
ff1['  '] = ''
ff1['sensitivity(0.6)(True Tag)'] = sen6[col6[0]]
ff1['sensitivity(0.6)(#text each Tag)'] = sen6[col6[1]]
ff1['precision(0.6)(True Tag)'] = sen6[col6[3]]
ff1['precision(0.6)(above cutoff)'] = sen6[col6[4]]
ff1['sensitivity(0.7)(True Tag)'] = sen7[col7[0]]
ff1['sensitivity(0.7)(#text each Tag)'] = sen7[col7[1]]
ff1['precision(0.7)(True Tag)'] = sen7[col7[3]]
ff1['precision(0.7)(above cutoff)'] = sen7[col7[4]]
ff1['sensitivity(0.8)(True Tag)'] = sen8[col8[0]]
ff1['sensitivity(0.8)(#text each Tag)'] = sen8[col8[1]]
ff1['precision(0.8)(True Tag)'] = sen8[col8[3]]
ff1['precision(0.8)(above cutoff)'] = sen8[col8[4]]
"""
ff1.to_csv('st_result5.csv', encoding='utf-8-sig')

In [13]:
print('1:Sentence_transformers +tuning:',len(ff1[ff1['tag']== ff1['tag1']]),'/',len(ff1),
      ', 정확도: ', round(len(ff1[ff1['tag']== ff1['tag1']])/len(ff1)*100,2) ,'%')
print('1:Sentence_transformers:       ',len(ff1[ff1['tag']== ff1['tag2']]),'/',len(ff1),
      ', 정확도: ', round(len(ff1[ff1['tag']== ff1['tag2']])/len(ff1)*100,2) ,'%')
print('2:TF-IDF:                      ',len(ff1[ff1['tag']== ff1['tag3']]),'/',len(ff1),
      ', 정확도: ', round(len(ff1[ff1['tag']== ff1['tag3']])/len(ff1)*100,2) ,'%')

1:Sentence_transformers +tuning: 1404 / 1718 , 정확도:  81.72 %
1:Sentence_transformers:        1352 / 1718 , 정확도:  78.7 %
2:TF-IDF:                       1166 / 1718 , 정확도:  67.87 %


In [14]:
ff2 =pd.DataFrame(pa_ac)
#ff2.to_csv('./drive/MyDrive/Colab_Notebooks/news/pair_result.csv', encoding='utf-8-sig')
#민감도 cut을 넘긴 텍스트중 tag가 맞는것/ 실제 tag에서 문장 개수
#정밀도 cut을 넘긴 텍스트중 tag가 맞는것/ cut을 넘긴 텍스트 개수
ff2.columns = ['text1', 'text2', 'tag1', 'tag2', 'm1_score', 'm2_score', 'm3_score']
ff2

,text1,text2,tag1,tag2,m1_score,m2_score,m3_score
0,바다가 일자리다…'2021 해양수산 온라인 취업박람회' 30일 개막,"일자리박람회 오프라인 면접, 다음 달 시작",1,1,0.4706,0.3961,0.0000
1,바다가 일자리다…'2021 해양수산 온라인 취업박람회' 30일 개막,경북 온·오프라인 채용박람회 개최,1,1,0.4869,0.4021,0.0000
2,바다가 일자리다…'2021 해양수산 온라인 취업박람회' 30일 개막,강원일자리박람회 내달 2일 대면 면접 돌입,1,1,0.3159,0.304,0.0000
3,바다가 일자리다…'2021 해양수산 온라인 취업박람회' 30일 개막,"안양시, 온·오프라인 연계 릴레이 일자리박람회",1,1,0.4968,0.4265,0.0000
4,바다가 일자리다…'2021 해양수산 온라인 취업박람회' 30일 개막,"‘경북 채용박람회’ 개최, 대면·비대면 병행 진행",1,1,0.4345,0.363,0.0000
...,...,...,...,...,...,...,...
1474898,희망회복자금 2차 신속지급 개시…61만여곳에 1조원 지원,희망회복자금 2차 신속지급 30일 개시…61만개사·1조원 규모,184,184,0.8128,0.8007,0.6843
1474899,희망회복자금 2차 신속지급 개시…61만여곳에 1조원 지원,"경남도, 소상공인 '희망복지자금' 2차 신속지급 시작",184,184,0.5929,0.6031,0.6001
1474900,희망회복자금 2차 지급 시작…61.1만개사에 1조 지원,희망회복자금 2차 신속지급 30일 개시…61만개사·1조원 규모,184,184,0.6738,0.6825,0.4138
1474901,희망회복자금 2차 지급 시작…61.1만개사에 1조 지원,"경남도, 소상공인 '희망복지자금' 2차 신속지급 시작",184,184,0.4255,0.4879,0.4769


In [21]:
t_ff = ff2[ff2['tag1'] == ff2['tag2']] 
f_ff = ff2[ff2['tag1'] != ff2['tag2']] 
f_ff = f_ff.sample(n= len(t_ff))
ff2 = t_ff.append(f_ff, ignore_index=True)
res = []
cut = 0.6
res.append([0,
            len(ff2[ff2['m1_score'] >=cut]),len(ff2[ff2['m1_score'] < cut]),
            len(ff2[ff2['m2_score'] >=cut]),len(ff2[ff2['m2_score'] < cut]),
            len(ff2[ff2['m3_score'] >=cut]),len(ff2[ff2['m3_score'] < cut])])
           
fff = ff2[ff2['tag1'] == ff2['tag2']]
res.append([len(fff),
            len(fff[fff['m1_score'] >=cut]),len(fff[fff['m1_score'] < cut]),
            len(fff[fff['m2_score'] >=cut]),len(fff[fff['m2_score'] < cut]),
            len(fff[fff['m3_score'] >=cut]),len(fff[fff['m3_score'] < cut])])
fff = ff2[ff2['tag1'] != ff2['tag2']]
res.append([len(fff),
            len(fff[fff['m1_score'] >=cut]),len(fff[fff['m1_score'] < cut]),
            len(fff[fff['m2_score'] >=cut]),len(fff[fff['m2_score'] < cut]),
            len(fff[fff['m3_score'] >=cut]),len(fff[fff['m3_score'] < cut])])
cut = 0.7
res.append([0,
            len(ff2[ff2['m1_score'] >=cut]),len(ff2[ff2['m1_score'] < cut]),
            len(ff2[ff2['m2_score'] >=cut]),len(ff2[ff2['m2_score'] < cut]),
            len(ff2[ff2['m3_score'] >=cut]),len(ff2[ff2['m3_score'] < cut])])
fff = ff2[ff2['tag1'] == ff2['tag2']]
res.append([len(fff),
            len(fff[fff['m1_score'] >=cut]),len(fff[fff['m1_score'] < cut]),
            len(fff[fff['m2_score'] >=cut]),len(fff[fff['m2_score'] < cut]),
            len(fff[fff['m3_score'] >=cut]),len(fff[fff['m3_score'] < cut])])
fff = ff2[ff2['tag1'] != ff2['tag2']]
res.append([len(fff),
            len(fff[fff['m1_score'] >=cut]),len(fff[fff['m1_score'] < cut]),
            len(fff[fff['m2_score'] >=cut]),len(fff[fff['m2_score'] < cut]),
            len(fff[fff['m3_score'] >=cut]),len(fff[fff['m3_score'] < cut])])
cut = 0.8
res.append([0,
            len(ff2[ff2['m1_score'] >=cut]),len(ff2[ff2['m1_score'] < cut]),
            len(ff2[ff2['m2_score'] >=cut]),len(ff2[ff2['m2_score'] < cut]),
            len(ff2[ff2['m3_score'] >=cut]),len(ff2[ff2['m3_score'] < cut])])
fff = ff2[ff2['tag1'] == ff2['tag2']]
res.append([len(fff),
            len(fff[fff['m1_score'] >=cut]),len(fff[fff['m1_score'] < cut]),
            len(fff[fff['m2_score'] >=cut]),len(fff[fff['m2_score'] < cut]),
            len(fff[fff['m3_score'] >=cut]),len(fff[fff['m3_score'] < cut])])
fff = ff2[ff2['tag1'] != ff2['tag2']]
res.append([len(fff),
            len(fff[fff['m1_score'] >=cut]),len(fff[fff['m1_score'] < cut]),
            len(fff[fff['m2_score'] >=cut]),len(fff[fff['m2_score'] < cut]),
            len(fff[fff['m3_score'] >=cut]),len(fff[fff['m3_score'] < cut])])
pd.DataFrame(res)

,0,1,2,3,4,5,6
0,0,4586,15794,4541,15839,3001,17379
1,10190,4579,5611,4536,5654,2997,7193
2,10190,7,10183,5,10185,4,10186
3,0,3037,17343,2801,17579,2015,18365
4,10190,3035,7155,2800,7390,2012,8178
5,10190,2,10188,1,10189,3,10187
6,0,1545,18835,1262,19118,1199,19181
7,10190,1545,8645,1262,8928,1198,8992
8,10190,0,10190,0,10190,1,10189


In [16]:
print('Cut off: 0.6')

print('1:Sentence_transformers +tuning:',len(ff2[(ff2[2]== ff2[3])]),'/',len(ff2),
      ', 정확도: ', round(len(ff2[(ff2[2]== ff2[3])& (ff2[4] >0.6)])/len(ff2)*100,2) ,'%')
print('2:Sentence_transformers:        ',len(ff2[(ff2[2]== ff2[3])]),'/',len(ff2),
      ', 정확도: ', round(len(ff2[(ff2[2]== ff2[3])& (ff2[4] >0.6)])/len(ff2)*100,2) ,'%')
print('3:TF-IDF:                       ',len(ff2[(ff2[2]== ff2[3])]),'/',len(ff2),
      ', 정확도: ', round(len(ff2[(ff2[2]== ff2[3])& (ff2[4] >0.6)])/len(ff2)*100,2) ,'%')

print('Cut off: 0.7')
print('1:Sentence_transformers +tuning:',len(ff2[(ff2[2]== ff2[3])& (ff2[4] >0.7)]),'/',len(ff2),
      ', 정확도: ', round(len(ff2[(ff2[2]== ff2[3])& (ff2[4] >0.6)])/len(ff2)*100,2) ,'%')
print('2:Sentence_transformers:        ',len(ff2[(ff2[2]== ff2[3])& (ff2[5] >0.7)]),'/',len(ff2),
      ', 정확도: ', round(len(ff2[(ff2[2]== ff2[3])& (ff2[4] >0.6)])/len(ff2)*100,2) ,'%')
print('3:TF-IDF:                       ',len(ff2[(ff2[2]== ff2[3])& (ff2[6] >0.7)]),'/',len(ff2),
      ', 정확도: ', round(len(ff2[(ff2[2]== ff2[3])& (ff2[4] >0.6)])/len(ff2)*100,2) ,'%')

print('Cut off: 0.8')
print('1:Sentence_transformers +tuning:',len(ff2[(ff2[2]== ff2[3])& (ff2[4] >0.8)]),'/',len(ff2),
      ', 정확도: ', round(len(ff2[(ff2[2]== ff2[3])& (ff2[4] >0.6)])/len(ff2)*100,2) ,'%')
print('2:Sentence_transformers:        ',len(ff2[(ff2[2]== ff2[3])& (ff2[5] >0.8)]),'/',len(ff2),
      ', 정확도: ', round(len(ff2[(ff2[2]== ff2[3])& (ff2[4] >0.6)])/len(ff2)*100,2) ,'%')
print('3:TF-IDF:                       ',len(ff2[(ff2[2]== ff2[3])& (ff2[6] >0.8)]),'/',len(ff2),
      ', 정확도: ', round(len(ff2[(ff2[2]== ff2[3])& (ff2[4] >0.6)])/len(ff2)*100,2) ,'%')

Cut off: 0.6


KeyError: 2